In [ ]:
%matplotlib inline

import ROOT, sys, os
from ROOT import std

from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches

from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

import root_numpy as rn
import pandas as pd

geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'

In [ ]:
reco_event_df = pd.DataFrame(rn.root2array("../reco_ana.root",treename='EventTree'))
reco_vtx3d_df = pd.DataFrame(rn.root2array("../reco_ana.root",treename='Vtx3DTree'))
reco_particle_df = pd.DataFrame(rn.root2array("../reco_ana.root",treename='ParticleTree'))
mc_df   = pd.DataFrame(rn.root2array("../mc_ana.root",treename='mctree'))

In [ ]:
#Calculating the signal denomenator...Looking for V-shape events
#1. Initial neutrino energy is in [100,500]MeV
#2. At least two particles in interaction
#3. Proton has energy > 80 Mev, dE/dx ~= 7.5Mev, ->20 wires on Y plane and 10 on UV plane.
#4. Energy Containment > 0.9, Select event with more thn 90% energy lie in crop Image

pdg2mass={}
pdg2mass[13]  = 105
pdg2mass[2212]= 938
pdg2mass[2112]= 939.5
pdg2mass[11]  = 0#0.5
pdg2mass[311] = 497.65
def mass(pdg):
    return pdg2mass[pdg]
mass_vec=np.vectorize(mass)

mc_wdf=mc_df.copy()
print mc_wdf.index.size
mc_wdf=mc_wdf.query('energyInit > 100 & energyInit < 500')
print mc_wdf.index.size
mc_wdf=mc_wdf[mc_wdf.daughterPdg_v.apply(lambda x : x.size>=2)]
print mc_wdf.index.size

def containment(row):
    pdg_v   = row['daughterPdg_v']
    einit_v = row['daughterEnergyInit_v']
    edep_v  = row['daughterEnergyDep_v']
    
    eisub_v=einit_v-mass_vec(pdg_v)
  
    init_sum = eisub_v.sum()
    dep_sum  = edep_v.sum()
    
    proton_v=eisub_v[np.where(pdg_v==2212)[0]]
    
    if(np.where(proton_v>80)[0].size>0): pass
    else: return False;
        
    if dep_sum / init_sum < 0.9: return False;

    return True;
    
mc_wdf=mc_wdf[mc_wdf.apply(containment,axis=1)]
print mc_wdf.index.size
mc_wdf_idx = mc_wdf.index
print mc_wdf_idx

In [ ]:
#Calculating the efficiency numerator
#reco_df_copy will have our analysis cuts
reco_event_wdf = reco_event_df.ix[mc_wdf_idx].copy()

cuts_v=[]
cuts_v.append('n_vtx3d>0')

for cut in cuts_v:
    reco_event_wdf = reco_event_wdf.query(cut)

print reco_event_wdf.index.size

#filter vertices from, only care about the events that have
#vertex estimated from 3 planes
#reco_wdf = reco_wdf[reco_wdf['num_planes_v'].apply(lambda x : np.where(x==3)[0].size)>0]

print reco_event_wdf.index.size

#cut on the pixel frac
#pxfrac=0.9
#def pixels(x):
#    y=np.hstack(x)
#    return np.where(y>pxfrac)[0].size

#reco_wdf=reco_wdf[reco_wdf['num_pixel_frac_vv'].apply(pixels) > 0]

In [ ]:
efficiency = 96/112.
print efficiency

In [ ]:

#join the event and vtx3d dataframes
#the event data frame has been filtered by number of 3D vertex
a=reco_event_wdf.set_index(['run','subrun','event'])
b=reco_vtx3d_df.set_index(['run','subrun','event'])
#get the vtx3d entries that are in events with 3D vertex
reco_df=a.join(b)
print reco_df.index.size


#group the event,run,and subrun into groups (1 group == 1 event now...)
event_group=reco_df.groupby(reco_df.index)

#for each group look at the num pixel frac, return the index in the group that gives highest SUM! -- this is vtx id
g=event_group['num_pixel_frac_v'].apply(lambda x : np.argmax(np.row_stack(x.values).sum(axis=1)))

top_idx=np.concatenate((np.vstack(g.index.values),g.values.reshape(g.values.shape[0],1)),axis=1)

#set the vertex ID on the index, this allows us to selection the correct r/s/e/vtxid
reco_df=reco_df.set_index('vtx3d_id',append=True,drop=True)

#get only the index if the highest pixel fraction
reco_df=reco_df.ix[pd.Index(top_idx).astype(np.object)]
print reco_df.index.size
#chosen reco dataframe (after analysis cuts)
reco_cdf = reco_df.reset_index(['vtx3d_id'])
print reco_df.index.size

In [ ]:
mc_cdf=mc_wdf.set_index(['run','subrun','event']).ix[reco_cdf.index]
print mc_wdf.index.size
print reco_cdf.index.size
dw_v=mc_cdf.vtx2d_w.values  - reco_cdf.circle_vtx_y_v.values
dt_v=mc_cdf.vtx2d_t.values  - reco_cdf.circle_vtx_x_v.values
#print dw_v

dw_v=np.vstack(dw_v)
dt_v=np.vstack(dt_v)
#print dw_v

In [ ]:
#print mc_wdf.loc[438]

In [ ]:
#print mc_wdf.index.size
sum_v       = []
dist_v      = []
lep_inie_v  = []
p_inie_v    = []
for entry in mc_wdf.index:
    thisrun    = mc_wdf.loc[entry].run
    thissubrun = mc_wdf.loc[entry].subrun
    thisevent  = mc_wdf.loc[entry].event
    for pdg,eng in zip(mc_wdf.loc[entry].daughterPdg_v,mc_wdf.loc[entry].daughterEnergyInit_v):
        if pdg == 13   : lep_inie_v.append(eng)
        if pdg == 2212 : p_inie_v.append(eng-938)
    thesevtx3d     = reco_vtx3d_df.query('(run==@thisrun) & (subrun == @thissubrun) & (event == @thisevent )')
    theserecoevent = reco_event_wdf.query('(run==@thisrun) & (subrun == @thissubrun) & (event == @thisevent )') 
    #print thesevtx3d
    thesevtx3d     = thesevtx3d.sort_values(by=['sum_pixel_frac'], ascending=False)
    if thesevtx3d.index.size >0 :
        d3dx = mc_wdf.loc[entry].parentX - thesevtx3d.iloc[0].vtx3d_x
        d3dy = mc_wdf.loc[entry].parentY - thesevtx3d.iloc[0].vtx3d_y
        d3dz = mc_wdf.loc[entry].parentZ - thesevtx3d.iloc[0].vtx3d_z
    
        d3d_v = np.sqrt(d3dx*d3dx + d3dy*d3dy + d3dz*d3dz)
    
        dist_v.append(d3d_v)
        sum_v.append(thesevtx3d.iloc[0].sum_pixel_frac)
        #if d3d_v > 50 and d3d_v <100  : 
            #print entry, d3d_v, thesevtx3d.iloc[0].sum_pixel_frac
            #print '   ', d3dx, d3dy,d3dz
            #print '   ', mc_wdf.loc[entry].parentX , thesevtx3d.iloc[0].vtx3d_x
        
plt.figure(figsize=(10,6))
plt.grid(True) # ALWAYS GRID TRUE
plt.hist2d(dist_v, sum_v ,[np.linspace(-1,72,100),np.linspace(0,3.5,100)])
plt.colorbar()
plt.title("highest tot num_pixel_frac : Vtx dist(MC-Reco[3d])", fontsize =15)

fig,ax=plt.subplots(figsize=(8,6))
a,_,_=plt.hist(lep_inie_v,bins=np.arange(0,500,20))
ax.set_ylim(0,20)
ax.grid()
ax.set_title('%i lepton Ini energy'%len(lep_inie_v))

fig,ax=plt.subplots(figsize=(8,6))
a,_,_=plt.hist(p_inie_v,bins=np.arange(0,500,20))
ax.set_ylim(0,40)
ax.grid()
ax.set_title('%i Proton Ini energy'%len(p_inie_v))


In [ ]:
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0)                                                                          
iom.add_in_file("../out_0000_0099.root")
iom.add_in_file("../ccqe_0100_0199.root")
iom.add_in_file("../ccqe_0200_0299.root")
iom.set_out_file("/tmp/trash.root")
iom.initialize()

larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("empty.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

In [ ]:
print mc_wdf.index.size
print mc_cdf.index.size
print reco_df.index.size
print reco_cdf.index.size
print reco_event_wdf.index.size
print reco_vtx3d_df.index.size

In [ ]:
faillist = []
ctr = 0
colormap=['red','blue','green','orange','pink','black']

for entry in mc_wdf.index:

    #print 'entry is ', entry
    thisrun    = mc_wdf.loc[entry].run
    thissubrun = mc_wdf.loc[entry].subrun
    thisevent  = mc_wdf.loc[entry].event
    #print thisrun, thissubrun, thisevent
    
    iom.read_entry(entry)
    larbysimg.process(iom)
    mgr=larbysimg.Manager()
    
    img_v = []
    
    fig,ax=plt.subplots(figsize=(20,20))
    maxx=0;minx=9999
    maxy=0;miny=9999
    plane=0
    caca_img=np.zeros((485, 485,3))
    cc=-1 #Add colors to different plane
    for mat in mgr.InputImages():
        cc+=1
        img_v.append(pygeo.image(mat))
        shape_img=np.where(img_v[plane]>1.5,1.0,0.0).astype(np.uint8)
        plane+=1
        nz_pixels=np.where(shape_img>0.0)
        
        maxx=np.maximum(maxx,np.max(nz_pixels[1])+100)
        minx=np.minimum(minx,np.min(nz_pixels[1])-100)

        maxy=np.maximum(maxy,np.max(nz_pixels[0])+100)
        miny=np.minimum(miny,np.min(nz_pixels[0])-100)

        caca_img[:,:,cc]=shape_img
    caca_img=np.where(caca_img>0.0,1.0,255)
    caca_img=caca_img.astype(np.uint8)
    plt.imshow(caca_img,interpolation='none')

    for iz in xrange(3):
        w_v=mc_wdf.loc[entry].vtx2d_w
        t_v=mc_wdf.loc[entry].vtx2d_t
        
        for w,t in zip(w_v,t_v):
            plt.plot(t,w,'^',color='yellow',markersize=15)
###MC info drawing ends here.
    thesevtx3d     = reco_vtx3d_df.query('(run==@thisrun) & (subrun == @thissubrun) & (event == @thisevent )')
    theserecoevent = reco_event_wdf.query('(run==@thisrun) & (subrun == @thissubrun) & (event == @thisevent )') 
    if theserecoevent.empty:    n_vtx3d        = 0
    else :                      
        n_vtx3d                             = theserecoevent.n_vtx3d
        thesevtx3d = thesevtx3d.sort_values(by=['sum_pixel_frac'], ascending=False)
    
    color_ctor = 0
    passflag = ' vtx found!'
    
    for x_v, y_v ,sum_f , x3d_v, y3d_v, z3d_v in zip(thesevtx3d.circle_vtx_x_v, 
                                                     thesevtx3d.circle_vtx_y_v , thesevtx3d.sum_pixel_frac,
                                                     thesevtx3d.vtx3d_x, thesevtx3d.vtx3d_y, thesevtx3d.vtx3d_z) :
        plt.plot(x_v,y_v,'*',color=colormap[color_ctor],markersize=20)
        plt.text(-100,0+color_ctor*40," sum_pixel_frac %0.3f \n Reco 3d vtx at x: %.1f,y: %.1f, z: %.1f"%
            (
             sum_f, x3d_v, y3d_v, z3d_v
            ),fontsize = 30, color = colormap[color_ctor])
            
        color_ctor+=1
    ax.set_title("Entry {}".format(entry),fontweight='bold',fontsize=40)
    ctr+=1

    mc_vtx_x       = mc_wdf.loc[entry].parentX
    mc_vtx_y       = mc_wdf.loc[entry].parentY
    mc_vtx_z       = mc_wdf.loc[entry].parentZ
    plt.text(-100,-50," %i,%s\n nutrino ini eng: %.0fMev \n MC 3d vtx at x: %.1f,y: %.1f, z: %.1f"%
             (
              n_vtx3d,passflag,
              mc_wdf.loc[entry].energyInit,
              mc_vtx_x,mc_vtx_y,mc_vtx_z,
             ),fontsize = 30)
       
    daupdg_v       = mc_wdf.loc[entry].daughterPdg_v
    dautrkid_v     = mc_wdf.loc[entry].daughterTrackid_v
    daupartrkid_v  = mc_wdf.loc[entry].daughterParentTrackid_v
    dauengini      = mc_wdf.loc[entry].daughterEnergyDep_v

    linespace = 0
    for pdg, trkid, ptrkid, engini in zip(daupdg_v, dautrkid_v, daupartrkid_v, dauengini):
        if trkid == ptrkid : 
            plt.text(-100,100+linespace*40," daughter PDG:\n %i, %.1fMeV"%
            (
            pdg,
            engini
            ),fontsize = 30)
            linespace +=1
    #plt.savefig("reco_%04d.png"%entry)
    #break
    plt.show()